In [51]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hdbscan
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
df=pd.read_csv('/content/drive/MyDrive/All_capped_keywords.csv')
titles = df['title']

Create the embeddings for all paper titles

In [53]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(titles, batch_size=64)
np.save("embeddings.npy", embeddings)

Approach #1 - Finding clusters

In [54]:
#Reducing embedding dimensions for better speed, but not needed
from sklearn.decomposition import PCA
embeddings = np.load("embeddings.npy")
pca = PCA(n_components=50)
embeddings_reduced = pca.fit_transform(embeddings)
np.save("embeddings_reduced50.npy", embeddings_reduced)

In [55]:
#Find the clusters in the reduced embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean')
embeddings_reduced = np.load("embeddings_reduced50.npy")
labels = clusterer.fit_predict(embeddings_reduced)
np.save("labels50.npy", labels)
labels = np.load("labels50.npy")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Creating semantic clusters at first seemed like a wise idea as I believed a cluster of silly / funny titles would've appeared in one of them, however this was not the case. I realized I may have to change gears and have to nudge the "filtering" towards titles that I specifically want, which are the silly/funny ones.



---



Additionally, I tried looking into seeing if the highest outliers in the embeddings were kinds of titles I was looking for, however it was not the case. The outliers just mean that they were simply very different than other titles and don't share any similarity, which does not imply they will turn out silly. The outliers may be unique, however it will allow the titles to only contain lots of technical jargon, which is the the opposite of what I wanted.


In [56]:
t10 = np.argsort(clusterer.outlier_scores_)[:10]
for i in t10:
  print(titles[i])

Seeking Common but Distinguishing Difference, A Joint Aspect-based Sentiment Analysis Model
Stealth-Persist: Architectural Support for Persistent Applications in Hybrid Memory Systems
PeriScope: An Effective Probing and Fuzzing Framework for the Hardware-OS Boundary
Miniature Haptics: Experiencing Haptic Feedback through Hand-based and Embodied Avatars
QuAC: Question Answering in Context
SpecDoctor: Differential Fuzz Testing to Find Transient Execution Vulnerabilities
Learning Unsupervised Metaformer for Anomaly Detection
Scene Context-Aware Salient Object Detection
Declarative Question Answering over Knowledge Bases containing Natural Language Text with Answer Set Programming
Making Decision like Human: Joint Aspect Category Sentiment Analysis and Rating Prediction with Fine-to-Coarse Reasoning


Approach #2 - Cosine similarity w/ seeds

The second approach I tried out was picking out ideal titles from a random selection in the large set of titles. I looked for silly, unique, and interesting titles.

At first, I kept the entire title for these selected seed titles, however there were many titles found "similar" to the seeds that only contained technical words and nothing eye catching. Then, I chose to eliminate the technical portions from the seeds and only keep the interesting and silly portions to help nudge the filtered titles to my desired output, which did seem to help. I also added some hand-written seeds that I would see as eye catching in a paper title.


In [57]:
seeds = [
    "Goodbye Python. Goodbye Java.",
    "Adios array bounds checking and godspeed garbage collector",
    "Auf wiedersehen exceptions",
    "So long OO-programming… whatever you are",
    "Adieu inferred polymorphic types, higher-order functions, rich built-in data",
    "My Brain is a Bag of Spiking Neurons",
    "A Tale of Two Dropouts",
    "Do Androids Dream of Differentiable Sheep?",
    "Neural Networks Say the Darndest Things",
    "Herding Cats with Gradient Descent",
    "Revenge of the Overfitting",
    "Attack of the Vanishing Gradients",
    "Lord of the Embeddings",
    "Fifty Shades of Backpropagation",
    "Harry Potter and the Order of the Convolution",
    "Raiders of the Lost Gradient",
    "Mission Impossible: Generalization Protocol",
    "The Matrix Reparameterized",
    "Much Ado About Attention",
    "The Good, the Bad, and the Overfitted",
    "Catch Me If You GAN",
    "To Err is Human, to Overfit Divine",
    "Fast and Curious",
    "The Curious Case of the Exploding Gradient",
    "Don’t Overfit Me Like That",
    "Dude, Where’s My GPU?",
    "Lost in Latent Space",
    "How I Learned to Stop Worrying and Love the Noise",
    "Fear and Loathing in Optimization",
    "Alice in Embeddingland",
    "Lord of the Weights",
    "Detect Me If You Can",
    "Cookie Swap Party",
    "Four-Dimensional Shopping Mall",
    "Are Two Heads Better Than One?",
    "Twin Peaks",
    "It's Almost Like They’re Trying to Hide It",
    "One Picture is Worth a Thousand Words?",
    "I Would Not Plant Apple Trees If the World Will Be Wiped",
    "The One Hundred Year Web",
    "The Chameleon Attack",
    "The Paradigm-Shift of Social Spambots",
    "Are All Successful Communities Alike?",
    "Fighting Against Deepfake: Patch & Pair",
    "The Spread of Physical Activity Through Social Networks",
    "FastSNG: The Fastest Social Network Dataset Generator",
    "Plumber",
    "Genre Differences of Song Lyrics",
    "Aspect of Blame",
    "Dude, Where’s My Gradient?"
]
print(len(seeds))


#Finding the embeddings for the seeds
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(seeds, batch_size=64, show_progress_bar=True)
np.save("seedEmbedding.npy", embeddings)

50


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
from sentence_transformers.util import cos_sim
import torch
all_embeddings = np.load("embeddings.npy")
seed_embeddings = np.load("seedEmbedding.npy")
sims = cos_sim(all_embeddings, seed_embeddings)  # shape: [num_titles, num_seeds]
print(sims.shape)


torch.Size([91919, 50])


In [59]:
length = 0
#Finding the titles that have a similarity of above a threshold of 0.5
for t in range(sims.shape[0]):
    best_sim = torch.max(sims[t]).item()
    if best_sim >= 0.5:
        print(f"{titles[t]} | {best_sim:.3f}")
        length += 1

print(f"{length} titles.")

Memorization Weights for Instance Reweighting in Adversarial Training | 0.503
Invariant Information Bottleneck for Domain Generalization | 0.566
Online DR-Submodular Maximization: Minimizing Regret and Constraint Violation | 0.534
A Deeper Look at the Hessian Eigenspectrum of Deep Neural Networks and its Applications to Regularization | 0.516
The HSIC Bottleneck: Deep Learning without Back-Propagation | 0.536
Safe Online Convex Optimization with Unknown Linear Safety Constraints | 0.511
Dropout Model Evaluation in MOOCs | 0.547
Beyond Unfolding: Exact Recovery of Latent Convex Tensor Decomposition Under Reshuffling | 0.506
Certifying the True Error: Machine Learning in Coq with Verified Generalization Guarantees | 0.525
Learning Nonlinear Dynamics in Efficient, Balanced Spiking Networks Using Local Plasticity Rules | 0.506
Auto-Encoding Transformations in Reparameterized Lie Groups for Unsupervised Learning | 0.505
The Counterfactual NESS Definition of Causation | 0.503
Understanding C

**Conclusion**

There were still some unideal titles found in the output set, so there can still be more improvements made with more time spent into optimizing the filtering. With more time, I would look into trying to have a dataset of bad titles that contain only technical jargon and have the similarity checking aim to get closer with silly embeddings and farther away from similarity with technical-only titles.